In [1]:
from finance import *

In [2]:
# Parámetros para función plot
estilos = mpf.available_styles()
tipos_graficos_disponibles = ['candle', 'ohlc', 'line']

In [3]:
########################################
# Funciones auxiliares para Ipywidgets #
########################################

def pct_change1(ticker, start_date=str(date.today()-timedelta(days=365)), end_date=str(date.today())): #STR
    """
    Dado un ticker, calcula el retorno si hubieramos invertido en la apertura del mercado
    del día start_date hasta el cierre del día actual.
    Formato de fecha: YYYY-MM-DD
    """
    
    # Descargar datos históricos desde start_date hasta end_date
    data = get_prices(ticker, start=start_date, end=end_date)
    
    initial_price = data['Open'].iloc[0]  # Precio de apertura en start_date
    last_price = data['Close'].iloc[-1]  # Precio de cierre actual
    percent_change = ((last_price - initial_price) / initial_price)
    
    return round(percent_change, 4)

def cagr1(ticker, start_date=str(date.today() - timedelta(days=365)), end_date=str(date.today())): #DATE
    """
    Obtiene el CAGR (Compound Annual Growth Rate) de una empresa.
    Para ello, toma los años enteros de diferencia desde el start_date hasta end_date, redondeando hacia abajo.
    Si no se especifica año, solo obtiene el pct de cambio de un año
    Formato de fecha: YYYY-MM-DD
    """
    # Normalizar el formato de las fechas si son cadenas
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, "%Y-%m-%d").date() if len(start_date) == 10 else datetime.strptime(start_date, "%Y-%m-%d")
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, "%Y-%m-%d").date() if len(end_date) == 10 else datetime.strptime(end_date, "%Y-%m-%d")

    # Descargar datos históricos desde start_date hasta end_date
    data = get_prices(ticker, start=str(start_date), end=str(end_date))

    anyos = int(((end_date - start_date).days / 365))  # Años de diferencia, redondeando hacia abajo
    initial_price = data['Open'].iloc[0]  # Precio de apertura en start_date
    last_price = data['Close'].iloc[-1]  # Precio de cierre actual
    
    cagr = ((last_price / initial_price) ** (1 / anyos) - 1)
    return round(cagr, 4)

def plot2(ticker, start_date=str(date.today()-timedelta(days=365)), end_date=str(date.today()),title='', volume=False,bollinger=False,info=False,
         style=estilos, type=tipos_graficos_disponibles, adicionales = ['-','macd', 'rsi'], savefigaux='', savefig = False):
    """
    Dada una fecha de inicio y una fecha de fin, realiza el gráfico de la cotización dentro de ese período.
    Si no hay fecha de fin, toma el último día de cotización. 
    Otros atributos:
    - volume: boolean
    - style= consult mpf.available_styles(), 'yahoo' is at default,
    - title= str,
    - ylabel= str,
    - ylabel_lower=str, 
    - savefig=str,
    - type = lines/candles
    - sma = tuple
    - info= boolean, muestra por pantalla información general de la empresa
    - macd= boolean, muestra el MACD
    - rsi = boolean, muestra el RSI.
    - bollinger = boolean, muestra las bandas de bollinger
    NOTA SOBRE INDICADORES: Por convención, la presencia de un indicador excluirá los demás, puedes plotear varios llamando varias veces al método plot
    Orden: macd > rsi 
    Formato de fecha: YYYY-MM-DD
    """
    # Obtener los datos de cotización utilizando yfinance
    data = get_prices(ticker, start=start_date, end=end_date)

    # Normalización
    if title=='': title = f'Cotización de {ticker}'
    #if ylabel=='': ylabel='Price'
    #if ylabel_lower=='': ylabel_lower='Volume'

    # Adición de ténicos y graficación
    if adicionales == 'macd': 
        ap = add_macd(data)
        if bollinger: ap.extend(add_bbands(data))
        if savefig is False:
            mpf.plot(data, type=type, addplot=ap, volume=volume, volume_panel=2, style=style, title=title, ylabel='Price', ylabel_lower='Volume')
        else:
            mpf.plot(data, type=type, addplot=ap, volume=volume, volume_panel=2, style=style, title=title, ylabel='Price', ylabel_lower='Volume', savefig=savefigaux+'.png')
    elif adicionales == 'rsi':
        ap = add_rsi(data)
        if bollinger: ap.extend(add_bbands(data))
        if savefig is False:
            mpf.plot(data, type=type, addplot=ap, volume=volume, volume_panel=2, style=style, title=title, ylabel='Price', ylabel_lower='Volume')
        else:
            mpf.plot(data, type=type, addplot=ap, volume=volume, volume_panel=2, style=style, title=title, ylabel='Price', ylabel_lower='Volume', savefig=savefigaux+'.png')

    elif adicionales == '-':
        ap = []
        if bollinger:
            ap.extend(add_bbands(data))
        if savefig is False:
            mpf.plot(data, type=type, volume=volume, addplot=ap, style=style, title=title, ylabel='Price', ylabel_lower='Volume')
        else:
            mpf.plot(data, type=type, volume=volume, addplot=ap, style=style, title=title, ylabel='Price', ylabel_lower='Volume', savefig=savefigaux+'.png')
    if info:
        load_repr_info(ticker)

#####################################

def loc_empresa(txt):
    """
    Función auxiliar que recibe un string y busca si es substring de los tickers que hay en tickers.csv
    """
    return [empr for empr in tickers if txt.lower() in empr.lower()]

def on_date_change(selected_date):
    """
    Función auxiliar para cambiar la fecha en el widget y limpiar la anterior interacción
    """
    clear_output(wait=True)

In [4]:
###############################
# Lista con varios los tickers #
###############################

tickers = []

with open('tickers.csv', 'r') as file:
    csvfile = csv.reader(file, delimiter=',')
    for l in csvfile:
        primer = l[0]
        tickers.append(primer[2:-1])

del tickers[0]

In [5]:
def ejecutar_accion(Funciones):
    if Funciones == 'Plot':
        @widgets.interact
        def manejar_empresa(txt='AAPL'):
            empresas = loc_empresa(txt)
            widgets.interact(
                plot2,
                ticker = empresas);

    elif Funciones == 'Pct_Change':
        @widgets.interact
        def manejar_empresa(txt='AAPL'):
            empresas = loc_empresa(txt)
            widgets.interact(
                pct_change1,
                ticker = empresas);

    elif Funciones == 'Cagr':
        @widgets.interact
        def manejar_empresa(txt='AAPL'):
            empresas = loc_empresa(txt)
            widgets.interact(
                cagr1,
                ticker = empresas);
            

In [6]:
widgets.interact(
    ejecutar_accion, 
    Funciones = ['-','Plot', 'Pct_Change', 'Cagr']
)

interactive(children=(Dropdown(description='Funciones', options=('-', 'Plot', 'Pct_Change', 'Cagr'), value='-'…

<function __main__.ejecutar_accion(Funciones)>